<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark/misc/read_from_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read from API
- Read data from API using requests and RDDs

# Setting up PySpark

In [ ]:
%pip install pyspark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').config('spark.ui.port', '4050').getOrCreate()
sc = spark.sparkContext

# Get data from API - Vehicles

In [18]:
import requests
from pyspark.sql.types import *

def readFromAPI(url: str, schema: StructType = None):
  response = requests.get(url)
  rdd = sc.parallelize(response.json())

  if schema:
    df = spark.read.schema(schema).json(rdd)
  else:
    df = spark.read.json(rdd)
  return df

In [ ]:
readFromAPI("https://api.carrismetropolitana.pt/municipalities").show()

# Vehicles

In [47]:
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])

vehicles = readFromAPI("https://api.carrismetropolitana.pt/vehicles", vehicle_schema)
print(vehicles.count())
vehicles.show()

847
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|     81|20241114-64010081...| IN_TRANSIT_TO|44|12096| 38.66496|   4503| -8.92511|  4503_0_2|  4503_0|            SCHEDULED|121640234560|      7.5| 100310|2024-11-14 16:47:47|4503_0_2|1400|163...|
|     52|20241114-64010572...|   INCOMING_AT|44|12672|38.710854|   4704|-8.925656|  4704_0_1|  4704_0|            SCHEDULED|123380234560|10.833333| 100433|2024-11-14 16:47:57|4704_0_1|1400|164...|
|    197|20

# Routes

In [43]:
from pyspark.sql.functions import *
routes = readFromAPI("https://api.carrismetropolitana.pt/routes")
print(routes.count())
routes.filter(array_contains(col("localities"), "Colégio Militar") & array_contains(col("localities"), "Brandoa")).show(100, False)

1017
+---------------+-------+----------+------+-------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------+--------------+--------------------+----------+----------+
|_corrupt_record|color  |facilities|id    |line_id|localities                                                                               |long_name                                                         |municipalities|patterns            |short_name|text_color|
+---------------+-------+----------+------+-------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------+--------------+--------------------+----------+----------+
|NULL           |#C61D23|[]        |1706_0|1706   |[Colégio Militar, Lisboa, Amadora, Brandoa, Casal da Mira, Alfornelos, Benfica]          |Colégio Militar (Metro) - UBBO via Brandoa           

# Questions

# Q1
- adjust ROUTE dataframe
  - use correct schema to get the data (StructField)
  - make sure _corrupt_record is removed from schema
- find the routes that pass through the localities "Brandoa" and "Colégio Militar"

### API - https://github.com/carrismetropolitana/api